In [1]:
#from module_1.homework import response
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-06-04 19:52:40--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2025-06-04 19:52:41 (10.8 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [5]:
with open('documents.json', 'rt') as f_in:
    doc_raw = json.load(f_in)

In [6]:
documents = []

for course_dict in doc_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [9]:
SELECT * WHERE course = 'data-engineering-zoomcamp';

SyntaxError: invalid syntax (580926268.py, line 1)

In [10]:
q = 'The course has already started, can I still enroll?'

In [11]:
index.fit(documents)

In [23]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict= {'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [24]:
from openai import OpenAI

In [25]:
client = OpenAI()


In [26]:
q

'The course has already started, can I still enroll?'

In [27]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [29]:
response.choices[0].message.content

"Whether you can still enroll in a course that has already started will depend on the institution or platform offering the course. Here are some steps you can take to find out:\n\n1. **Check the Course Website**: Look for any information about late enrollment policies on the course's official website or listing.\n\n2. **Contact the Instructor**: Reach out to the instructor or course coordinator to ask if they are willing to accept late enrollments.\n\n3. **Speak with the Admissions Office**: If it's a university or college course, contact the admissions office or registrar for more information.\n\n4. **Consider Auditing**: If you can't officially enroll, ask if it's possible to audit the course, which allows you to attend without receiving credit.\n\n5. **Look for Online Options**: Some online platforms may allow you to join classes at your own pace, even after the official start date.\n\nAlways act quickly, as there might be deadlines for adding courses late, and try to explain your c

In [36]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when answering the QUESTION. If the CONTEXT doesnt contain the answer, output NONE

QUESTION: {question}

CONTEXT: {context}
""".strip()

In [37]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [40]:
prompt = prompt_template.format(question=q, context=context).strip()

In [41]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when answering the QUESTION. If the CONTEXT doesnt contain the answer, output NONE

QUESTION: The course has already started, can I still enroll?

CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

s

In [42]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

"Yes, you can still enroll in the course after it has started. Even if you don't register right away, you're still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects, so it is advisable not to leave everything for the last minute."